# Optimization via Limited-memory BFGS and Differentiable Physics

In this notebook, we optimize the initial state of the buoyancy-driven flow with obstacles simulation to match the final state of a simulation. We use the L-BFGS optimizer and PhiFlow for differentiable physics to optimize the initial state of a simulation to match the final state of a simulation. 

### Setup imports and environment

In [8]:
import sys
import phi.torch

sys.path.append('github/smdp/buoyancy-flow') 

Load LPIPS for perceptual distance metric

In [9]:
# import LPIPS distance
import evaluation.lpips as lpips

Setup data from the test dataset

In [10]:
import h5py

# import dataloader
from dataloader_multi import DataLoader

file_test = 'github/smdp/buoyancy-flow/data/smoke_plumes_test_r0.h5'

dataKeys = None      
with h5py.File(file_test, 'r') as f:
    dataKeys = list(f.keys())

dataKeys = list(zip([file_test] * len(dataKeys), dataKeys))

test_data = DataLoader([file_test], dataKeys, name='test', batchSize=1)

Exception ignored in: <function DataLoader.__del__ at 0x7f3948c5e700>
Traceback (most recent call last):
  File "/home/benjamin/projects/github/smdp/buoyancy-flow/dataloader_multi.py", line 87, in __del__
    self.h5files.close()
AttributeError: 'dict' object has no attribute 'close'


Length: 5


### Optimization with L-BFGS
First, we define the time of the initial state that we reconstruct (t=0.35) and the number of optimization steps of the L-BFGS optimizer (10). 

In [11]:
time_init = 35 # t=0.35 
optimization_steps = 10

params = {
    'batch_size' : 1,
    'DT' : 0.01,
    't1': 0.65,
    'time_init': time_init,
    'optimization_steps': optimization_steps,
}

In [12]:
from physics_check import batch_inflow, physics_forward, batch_geometries_pre_phiflow
from eval import eval_forward   
from phi.torch.flow import *
from tqdm import tqdm

phi.math.backend.set_global_default_backend(phi.torch.TORCH) 

def optimize_sample(item, params):
    
    time_init = params['time_init'] 
    optimization_steps = params['optimization_steps']   
    
    simulation_metadata = {}
    simulation_metadata['NSTEPS'] = int(params['t1'] / params['DT'])
    simulation_metadata['INFLOW'] = batch_inflow(item['INFLOW'], batchSize=params['batch_size'])
    simulation_metadata['INFLOW_1b'] = batch_inflow(item['INFLOW'], batchSize=1)
    bounds = item['BOUNDS']
    simulation_metadata['BOUNDS'] = Box(x=(bounds['_lower'][0], bounds['_upper'][0]),
                                        y=(bounds['_lower'][1], bounds['_upper'][1]))
    simulation_metadata['smoke_res'] = item['smoke_res']
    simulation_metadata['v_res'] = item['v_res']
    simulation_metadata['DT'] = params['DT']

    obstacles = [item['obstacle_list']]
    obstacles = batch_geometries_pre_phiflow(obstacles)

    smoke_state = torch.asarray(item['smoke'], dtype=torch.float32)
    vel_x_state = torch.asarray(item['vel_x'], dtype=torch.float32)
    vel_y_state = torch.asarray(item['vel_y'], dtype=torch.float32)
    mask_state = torch.asarray(item['mask'], dtype=torch.float32)

    init_state = [smoke_state[0][None], vel_x_state[0][None], vel_y_state[0][None], mask_state[0][None]]
    target_state = [smoke_state[-1][None].to('cuda:0'), vel_x_state[-1][None].to('cuda:0'),
                    vel_y_state[-1][None].to('cuda:0')]

    zero_smoke = torch.zeros_like(init_state[0]).to('cuda:0').clone().detach().requires_grad_(True)
    zero_vel_x = torch.zeros_like(init_state[1]).to('cuda:0').clone().detach().requires_grad_(True)
    zero_vel_y = torch.zeros_like(init_state[2]).to('cuda:0').clone().detach().requires_grad_(True)

    forward_fn = physics_forward(simulation_metadata)
    forward_fn = math.jit_compile(forward_fn)

    _ = eval_forward(init_state, obstacles, simulation_metadata, physics_forward_fn=forward_fn, t0=0.0)

    def loss_function(init_state_):

        t0 = simulation_metadata['DT'] * time_init

        init_state_.append(torch.zeros_like(init_state[3]).clone().detach().requires_grad_(False))

        simulation_metadata['NSTEPS'] = int((params['t1'] - t0) / params['DT'])

        out = eval_forward(init_state_, obstacles, simulation_metadata, physics_forward_fn=forward_fn, t0=t0)

        smoke_out = out[-1][0][0]
        vel_x_out = out[-1][1][0]
        vel_y_out = out[-1][2][0]

        smoke_target = target_state[0][0]
        vel_x_target = target_state[1][0]
        vel_y_target = target_state[2][0]

        return torch.nn.functional.mse_loss(smoke_target, smoke_out) + torch.nn.functional.mse_loss(vel_x_target,
                                                                                                    vel_x_out) + torch.nn.functional.mse_loss(
            vel_y_target, vel_y_out)

    lbfgs = optim.LBFGS([zero_smoke, zero_vel_x, zero_vel_y],
                        history_size=10,
                        max_iter=4,
                        line_search_fn="strong_wolfe")


    def closure():
        lbfgs.zero_grad()
        objective = loss_function([zero_smoke, zero_vel_x, zero_vel_y])
        objective.backward()
        return objective


    pbar = tqdm(range(optimization_steps))

    for _ in pbar:
        loss = loss_function([zero_smoke, zero_vel_x, zero_vel_y]).item()
        lbfgs.step(closure)
        pbar.set_description("Loss: %s" % loss)

    prediction = eval_forward([zero_smoke, zero_vel_x, zero_vel_y, init_state[3]], obstacles, simulation_metadata, physics_forward_fn=forward_fn, t0=simulation_metadata['DT'] * time_init)
    
    return [(marker_field.detach().cpu().numpy(), vel_x_field.detach().cpu().numpy(), 
             vel_y_field.detach().cpu().numpy(), mask_field.detach().cpu().numpy()) 
            for marker_field, vel_x_field, vel_y_field, mask_field in prediction]

In [13]:
results = {}

reconstruction_MSE = 0
lpips_smoke = 0

for key in dataKeys:

    item = test_data.load(key)

    prediction = optimize_sample(item, params)

    results[key] = prediction   

    smoke_state = torch.asarray(item['smoke'], dtype=torch.float32)

    reconstruction_MSE += torch.nn.functional.mse_loss(torch.tensor(prediction[-1][0][0]), smoke_state[-1]).item()

    lpips_smoke += lpips.lpips_dist(prediction[0][0], smoke_state[time_init][None].numpy())

lpips_smoke = lpips_smoke / len(dataKeys)
reconstruction_MSE = reconstruction_MSE / len(dataKeys)

print('Reconstruction MSE: ', reconstruction_MSE)
print('LPIPS smoke: ', lpips_smoke)

jit compile physics
tracing physics forwards...
tracing physics forwards...


Loss: 0.23229913413524628: 100%|████████████████| 10/10 [04:13<00:00, 25.31s/it]


jit compile physics
tracing physics forwards...
tracing physics forwards...


Loss: 0.33518481254577637: 100%|████████████████| 10/10 [03:44<00:00, 22.45s/it]


jit compile physics
tracing physics forwards...
tracing physics forwards...


...

jit compile physics
tracing physics forwards...
tracing physics forwards...


Loss: 0.1765131950378418: 100%|█████████████████| 10/10 [05:38<00:00, 33.89s/it]


jit compile physics
tracing physics forwards...
tracing physics forwards...


Loss: 0.27883967757225037: 100%|████████████████| 10/10 [04:03<00:00, 24.32s/it]


Reconstruction MSE:  0.20247265696525574
LPIPS smoke:  0.543474805355072


### Save results

In [14]:
# save results to file
import pickle   

results_file = 'github/smdp/buoyancy-flow/evaluation/results/results_lbfgs.pkl'

with open(results_file, 'wb') as f:
    pickle.dump(results, f)